In [1]:
from transformers import AutoTokenizer, AutoModel
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import layers
import torch
import numpy as np

2024-03-18 12:25:16.954706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 12:25:16.954849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 12:25:17.128365: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def getLexedArrays(path):
    array = []
    with open(path, 'r') as f:
        while True:
            line = f.readline()
            if line:
                array.append(line.split())
            else:
                break
    return array


def getLabels(path):
    array = []
    with open(path, 'r') as f:
        while True:
            line = f.readline()
            if line:
                array.append(int(line))
            else:
                break
    return array

In [3]:
test_arrays = getLexedArrays('../input/lexed-vdisc/lexed_VDISC/VDISC_test')
train_arrays = getLexedArrays('../input/lexed-vdisc/lexed_VDISC/VDISC_train')
val_arrays = getLexedArrays('../input/lexed-vdisc/lexed_VDISC/VDISC_validate')

test_labels = getLabels('../input/labels-vdisc/labels_VDISC/VDISC_test')
train_labels = getLabels('../input/labels-vdisc/labels_VDISC/VDISC_train')
val_labels = getLabels('../input/labels-vdisc/labels_VDISC/VDISC_validate')

#### **Pad length of arrays with Zeroes**

In [16]:
from tqdm.notebook import tqdm

max_len = 0
for lexed_arrays in (test_arrays, train_arrays, val_arrays):
    for ar in lexed_arrays:
        max_len = max(max_len, len(ar))

npars = []
for lexed_arrays in (test_arrays, train_arrays, val_arrays):            
    npar = np.zeros((len(lexed_arrays), max_len))
    for i, ar in enumerate(tqdm(lexed_arrays)):
        A = np.array(ar)
        if A.size < max_len:
            npar[i, :A.size] = A 
        else:
            npar[i, :] = A[:max_len] 
        
    npars.append(npar)
    
test_array, train_array, val_array = npars

  0%|          | 0/127419 [00:00<?, ?it/s]

  0%|          | 0/1019471 [00:00<?, ?it/s]

  0%|          | 0/127476 [00:00<?, ?it/s]

In [ ]:
# Note to self: do not run or your memory runs out
test_ds = tf.data.Dataset.from_tensor_slices((test_array, test_labels))
train_ds = tf.data.Dataset.from_tensor_slices((train_array, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_array, val_labels))

In [7]:
class BERTLayer(layers.Layer):
    def __init__(self, name="microsoft/codebert-base", **kwargs):
        super(BERTLayer, self).__init__(**kwargs)
        self.tokenizer = AutoTokenizer.from_pretrained(name)
        self.model = AutoModel.from_pretrained(name)
        self.name = name

    def call(self, inputs):
        inputs = np.array(inputs).astype(str)
        tokeneized = self.tokenizer.convert_tokens_to_ids(inputs)[0]
        
        inputs = tf.cast(tokenized, dtype=tf.int32)
        inputs_pt = torch.tensor(inputs.numpy(), dtype=torch.long)
        with torch.no_grad():
            outputs = self.model(inputs_pt)[0]
        outputs_tf = tf.convert_to_tensor(outputs.numpy(), dtype=tf.float32)
        return outputs_tf

In [11]:
model = Sequential([
    BERTLayer(),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense()
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


NameError: name 'num_classes' is not defined

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)